# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 25 2022 11:17AM,237992,16,SAR00000333,Sartorius Stedim North America,Released
1,May 25 2022 10:55AM,238059,10,MET210520,"Methapharm, Inc.",Released
2,May 25 2022 10:55AM,238059,10,MET210538,"Methapharm, Inc.",Released
3,May 25 2022 10:55AM,238059,10,MET210539,"Methapharm, Inc.",Released
4,May 25 2022 10:55AM,238059,10,MET210549,"Methapharm, Inc.",Released
5,May 25 2022 10:55AM,238059,10,MET210551,"Methapharm, Inc.",Released
6,May 25 2022 10:53AM,238057,10,8057573,Eywa Pharma Inc.,Released
7,May 25 2022 10:53AM,238057,10,8057574,Eywa Pharma Inc.,Released
8,May 25 2022 10:53AM,238057,10,8057616,Eywa Pharma Inc.,Released
9,May 25 2022 10:45AM,238056,19,60016873,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,238052,Released,42
35,238053,Released,1
36,238056,Released,1
37,238057,Released,3
38,238059,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238052,NaN,NaN,42.0
238053,NaN,NaN,1.0
238056,NaN,NaN,1.0
238057,NaN,NaN,3.0
238059,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237965,0.0,0.0,1.0
237975,8.0,8.0,0.0
237978,2.0,6.0,0.0
237980,0.0,0.0,1.0
237981,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237965,0,0,1
237975,8,8,0
237978,2,6,0
237980,0,0,1
237981,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237965,0,0,1
1,237975,8,8,0
2,237978,2,6,0
3,237980,0,0,1
4,237981,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237965,,,1
1,237975,8,8,
2,237978,2,6,
3,237980,,,1
4,237981,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 25 2022 11:17AM,237992,16,Sartorius Stedim North America
1,May 25 2022 10:55AM,238059,10,"Methapharm, Inc."
6,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.
9,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc."
10,May 25 2022 9:48AM,238021,20,"ACI Healthcare USA, Inc."
45,May 25 2022 9:35AM,238047,15,"Person & Covey, Inc."
51,May 25 2022 9:25AM,238052,10,ISDIN Corporation
93,May 25 2022 9:24AM,238053,10,ISDIN Corporation
94,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc."
95,May 25 2022 9:03AM,238041,15,"Laser Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 25 2022 11:17AM,237992,16,Sartorius Stedim North America,,,1
1,May 25 2022 10:55AM,238059,10,"Methapharm, Inc.",,,5
2,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.,,,3
3,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc.",,,1
4,May 25 2022 9:48AM,238021,20,"ACI Healthcare USA, Inc.",,8,27
5,May 25 2022 9:35AM,238047,15,"Person & Covey, Inc.",,3,3
6,May 25 2022 9:25AM,238052,10,ISDIN Corporation,,,42
7,May 25 2022 9:24AM,238053,10,ISDIN Corporation,,,1
8,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc.",,,1
9,May 25 2022 9:03AM,238041,15,"Laser Pharmaceuticals, LLC",,6,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 11:17AM,237992,16,Sartorius Stedim North America,1,,
1,May 25 2022 10:55AM,238059,10,"Methapharm, Inc.",5,,
2,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.,3,,
3,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc.",1,,
4,May 25 2022 9:48AM,238021,20,"ACI Healthcare USA, Inc.",27,8,
5,May 25 2022 9:35AM,238047,15,"Person & Covey, Inc.",3,3,
6,May 25 2022 9:25AM,238052,10,ISDIN Corporation,42,,
7,May 25 2022 9:24AM,238053,10,ISDIN Corporation,1,,
8,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc.",1,,
9,May 25 2022 9:03AM,238041,15,"Laser Pharmaceuticals, LLC",12,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 11:17AM,237992,16,Sartorius Stedim North America,1,,
1,May 25 2022 10:55AM,238059,10,"Methapharm, Inc.",5,,
2,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.,3,,
3,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc.",1,,
4,May 25 2022 9:48AM,238021,20,"ACI Healthcare USA, Inc.",27,8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 11:17AM,237992,16,Sartorius Stedim North America,1.0,NaN,NaN
1,May 25 2022 10:55AM,238059,10,"Methapharm, Inc.",5.0,NaN,NaN
2,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.,3.0,NaN,NaN
3,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,May 25 2022 9:48AM,238021,20,"ACI Healthcare USA, Inc.",27.0,8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 11:17AM,237992,16,Sartorius Stedim North America,1.0,0.0,0.0
1,May 25 2022 10:55AM,238059,10,"Methapharm, Inc.",5.0,0.0,0.0
2,May 25 2022 10:53AM,238057,10,Eywa Pharma Inc.,3.0,0.0,0.0
3,May 25 2022 10:45AM,238056,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,May 25 2022 9:48AM,238021,20,"ACI Healthcare USA, Inc.",27.0,8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1904362,78.0,64.0,2.0
15,1190201,32.0,18.0,0.0
16,476017,2.0,0.0,0.0
18,952007,6.0,17.0,0.0
19,952080,3.0,8.0,8.0
20,713988,27.0,48.0,16.0
21,237980,1.0,0.0,0.0
22,238050,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1904362,78.0,64.0,2.0
1,15,1190201,32.0,18.0,0.0
2,16,476017,2.0,0.0,0.0
3,18,952007,6.0,17.0,0.0
4,19,952080,3.0,8.0,8.0
5,20,713988,27.0,48.0,16.0
6,21,237980,1.0,0.0,0.0
7,22,238050,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,78.0,64.0,2.0
1,15,32.0,18.0,0.0
2,16,2.0,0.0,0.0
3,18,6.0,17.0,0.0
4,19,3.0,8.0,8.0
5,20,27.0,48.0,16.0
6,21,1.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,78.0
1,15,Released,32.0
2,16,Released,2.0
3,18,Released,6.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,2.0,0.0,0.0,0.0,8.0,16.0,0.0,0.0
Executing,64.0,18.0,0.0,17.0,8.0,48.0,0.0,0.0
Released,78.0,32.0,2.0,6.0,3.0,27.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,8.0,16.0,0.0,0.0
1,Executing,64.0,18.0,0.0,17.0,8.0,48.0,0.0,0.0
2,Released,78.0,32.0,2.0,6.0,3.0,27.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,2.0,0.0,0.0,0.0,8.0,16.0,0.0,0.0
1,Executing,64.0,18.0,0.0,17.0,8.0,48.0,0.0,0.0
2,Released,78.0,32.0,2.0,6.0,3.0,27.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()